In [1]:
import numpy as np
import keras
import os
import pandas as pd
import urllib
import random

np.random.seed(42)  # for reproducibility
from sklearn import metrics
from sklearn import metrics
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Embedding, LSTM, Input, Bidirectional, Lambda
import keras.backend as K  # Needed for max pooling operation
from resnet import Residual
from keras.layers.core import Dense, Activation, Flatten
from matplotlib import pyplot as plt
max_features = 5000
maxlen = 400

Using Theano backend.


In [2]:
# AUC measure
def auc_score(y_true, y_pred):
    return metrics.roc_auc_score(y_true, y_pred)

def plot_ROC(fpr, tpr, roc_auc):
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.savefig("Resnet-50layer.png")

In [3]:
# Read the data into a Pandas dataframe.
comments = pd.read_csv('attack_annotated_comments.tsv', sep = '\t', index_col = 0)
annotations = pd.read_csv('attack_annotations.tsv',  sep = '\t')

# Label a comment as an attack if over half of annotators did so.
labels = annotations.groupby('rev_id')['attack'].mean() > 0.5

# Join labels and comments
comments['attack'] = labels

# Preprocess the data -- remove newlines, tabs, quotes
# Something to consider: remove Wikipedia style markup (::'s and =='s)
comments['comment'] = comments['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
comments['comment'] = comments['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))
comments['comment'] = comments['comment'].apply(lambda x: x.replace("`", " "))

In [4]:
# Grab the training, dev, and test data (3:1:1 split)
train_data = comments.loc[comments['split'] == 'train']
dev_data = comments.loc[comments['split'] == 'dev']
test_data = comments.loc[comments['split'] == 'test']

# The list of gold-standard labels for the data
train_labels = train_data["attack"].tolist()
dev_labels = dev_data["attack"].tolist()
test_labels = test_data["attack"].tolist()
# Put all the training data (comments) into a list
train_texts = train_data["comment"].tolist()
dev_texts = dev_data["comment"].tolist()
test_texts = test_data["comment"].tolist()

In [5]:
def randomly_sample(comments, labels):
    # grab all the attacks to see how many we need to match
    attack_indices = [i for i in range(len(comments)) if labels[i] == True]
    new_training = [comments[i] for i in attack_indices]
    new_labels = [labels[i] for i in attack_indices]
    
    # grab all the ones that are not attacks, shuffle them and select the same number of non-attacks
    non_attack_indices = [i for i in range(len(comments)) if labels[i] == False]
    random.shuffle(non_attack_indices)
    for i in range(len(attack_indices)):
        new_training.append(comments[i])
        new_labels.append(labels[i])
        
    # shuffle the training and labeling so that they're still matched 1:1
    shuf_indices = [i for i in range(len(new_training))]
    random.shuffle(shuf_indices)
    shuffled_training = [new_training[i] for i in shuf_indices]
    shuffled_labels = [new_labels[i] for i in shuf_indices]
    
    return shuffled_training, shuffled_labels

In [6]:
# randomly select non-attack data such that it creates an even split with the attack data
train_texts, train_labels = randomly_sample(train_texts, train_labels)
dev_texts, dev_labels = randomly_sample(dev_texts, dev_labels)
test_texts, test_labels = randomly_sample(test_texts, test_labels)
print(len(train_texts), len(train_labels))

840 840


In [7]:
# The word level tokenizer using only top 5000 words
tokenizer = keras.preprocessing.text.Tokenizer(num_words=5000)

In [8]:
# Fit it to the training data.
tokenizer.fit_on_texts(train_texts)
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

dev_tokenizer =  keras.preprocessing.text.Tokenizer(num_words=5000)
dev_tokenizer.fit_on_texts(dev_texts)

test_tokenizer =  keras.preprocessing.text.Tokenizer(num_words=5000)
test_tokenizer.fit_on_texts(test_texts)


7987


In [9]:
# Transform each comment in the training data to arrays of equal length
train_seq = tokenizer.texts_to_sequences(train_texts)
dev_seq = tokenizer.texts_to_sequences(dev_texts)
test_seq = tokenizer.texts_to_sequences(test_texts)

# create an embedding layer of wordvec values
# assumes you have downloaded he glove.6b dataset here: http://nlp.stanford.edu/data/glove.6B.zip
embeddings_index = {}
f = open(os.path.join(str(os.getcwd())+"/glove.6B", 'glove.6B.200d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))


Found 400000 word vectors.


In [10]:
embedding_dim = 200
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [11]:
train_seq = sequence.pad_sequences(train_seq, maxlen=maxlen)
dev_seq = sequence.pad_sequences(dev_seq, maxlen=maxlen)
test_seq = sequence.pad_sequences(test_seq, maxlen=maxlen)

In [12]:
print(max_features, len(train_seq), embedding_matrix.shape[0], maxlen)

5000 840 7987 400


In [13]:
input = Input(shape=(maxlen,))
embedding_layer = Embedding(vocab_size,
                            embedding_dim,
                            weights=[embedding_matrix],
                            input_length=len(train_seq),
                            trainable=False)(input)

# you can fiddle with the number of layers within this function
def get_model():
    input = Input(shape=(maxlen, 200))
    output = Dense(200)(input)
    for _ in range(49):
        output = Dense(200)(output)
    return Model(input, output)
    

In [14]:
resnet = Residual(get_model())(embedding_layer)
maxpool = Lambda(lambda x: K.max(x, axis=1, keepdims=False),
                 output_shape=lambda x: (x[0], x[2]))(resnet)
dropout = Dropout(0.5)(maxpool)
output = Dense(1, activation='sigmoid')(dropout)
model = Model(input=input, output=output)

/Users/tracyrohlin/Documents/Masters_2.0/NeuralNet/Trolling_RNN/resnet.py:66: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  self.merge_mode = Merge(mode=self.merge_mode)
/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/keras/engine/topology.py:583: UserWarning: Class `resnet.Residual` defines `get_output_shape_for` but does not override `compute_output_shape`. If this is a Keras 1 layer, please implement `compute_output_shape` to support Keras 2.
  output_shape = self.compute_output_shape(input_shape)
/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=sigmoid.0, inputs=/input_1)`


In [15]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [16]:
model.fit(train_seq, train_labels,
          batch_size=100,
          epochs=4)

Epoch 1/4
840/840 [==============================] - 176s - loss: 6.2664 - acc: 0.5310     
Epoch 2/4
840/840 [==============================] - 155s - loss: 7.7631 - acc: 0.5131     

In [17]:

# Evaluate model
score, acc = model.evaluate(test_seq, test_labels, batch_size=128)
print('\nAccuracy: %1.4f' % acc)
print('Score: %1.4f' % score)


296/296 [==============================] - 11s    

Accuracy: 0.5101
Score: 7.8096


In [18]:


y_pred = model.predict(test_seq, batch_size=2)
roc = auc_score(test_labels, y_pred)
print('AUROC:{}'.format(roc))
fpr, tpr, _ = metrics.roc_curve(test_labels, y_pred)
    
plot_ROC(fpr, tpr, roc)

AUROC:0.5034246575342466
